In [1]:
import torch
from peft import LoraConfig, get_peft_model
from PIL import Image
from transformers import IdeficsForVisionText2Text, AutoProcessor, BitsAndBytesConfig
import torchvision.transforms as transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

checkpoint = "/home/gpuuser3/sinngam_albert/work/mllm_finetune/idefics_9B_MMSD_outputs_merged"

processor = AutoProcessor.from_pretrained(checkpoint)
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, device_map="auto")

IdeficsForVisionText2Text has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Instantiating IdeficsAttention without passing a `layer_idx` is not recommended and will lead to errors during the forward call if caching is used. Please

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [16]:
def check_inference(model, processor, prompts, max_new_tokens=50):
    tokenizer = processor.tokenizer
    bad_words = ["<image>", "<fake_token_around_image>"]
    if len(bad_words) > 0:
        bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids

    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    inputs = processor(prompts, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, eos_token_id=[eos_token_id], bad_words_ids=bad_words_ids, max_new_tokens=max_new_tokens, early_stopping=True)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    answer = generated_text.split("Answer:")[-1].strip()
    # check if answer is either "SARCASTIC" or "NOT SARCASTIC"
    if answer in ["SARCASTIC", "NOT SARCASTIC"]:
        return answer
    else:
        return "UNKNOWN"

In [17]:
url = "/home/gpuuser3/sinngam_albert/datasets/mmsd2/dataset_image/899600809389953024.jpg"
prompts = [
    Image.open(url),
    "Question: Classify the text <Kill me now> and the image into one of the following categories: <SARCASTIC, NOT SARCASTIC>. Answer:",
]
check_inference(model, processor, prompts, max_new_tokens=50)


'SARCASTIC'

In [22]:
import pandas as pd

mmsd_test = pd.read_json("/home/gpuuser3/sinngam_albert/datasets/mmsd2/test.json")
mmsd_test['image_path'] = mmsd_test['image_id'].apply(
    lambda x: f"/home/gpuuser3/sinngam_albert/datasets/mmsd2/dataset_image/{x}.jpg"
)

mmsd_test['question'] = mmsd_test['text'].apply(
    lambda x: f"Question: Classify the text <{x}> and the image into one of the following categories: <SARCASTIC, NOT SARCASTIC>. Answer:"
)

In [23]:
mmsd_test.head()

,image_id,text,label,image_path,question
0,862902619928506372,i am guessing # netflix no longer lets you gra...,1,/home/gpuuser3/sinngam_albert/datasets/mmsd2/d...,Question: Classify the text <i am guessing # n...
1,892551658487631873,it 's the insensitive strikeouts at suntrust p...,1,/home/gpuuser3/sinngam_albert/datasets/mmsd2/d...,Question: Classify the text <it 's the insensi...
2,853143461360480256,"following the path of the river calder , so .....",1,/home/gpuuser3/sinngam_albert/datasets/mmsd2/d...,Question: Classify the text <following the pat...
3,918423568823840768,# westernsahara # authority has no lessons 2ge...,1,/home/gpuuser3/sinngam_albert/datasets/mmsd2/d...,Question: Classify the text <# westernsahara #...
4,731617467718610944,hey <user> great sale !,1,/home/gpuuser3/sinngam_albert/datasets/mmsd2/d...,Question: Classify the text <hey <user> great ...


In [24]:
for idx, row in mmsd_test.iterrows():
    print(f"Processing row {idx+1}/{len(mmsd_test)}")
    
    response = check_inference(
        model,
        processor,
        [Image.open(row['image_path']), row['question']],
        max_new_tokens=50
    )
    print(f"Input=> {row['question']}\nResponse=> {response}\n")
    mmsd_test.at[idx, 'idefics2_response'] = response

mmsd_test['idefics2_label'] = mmsd_test['idefics2_response'].apply(
    lambda x: "0" if "NOT" in x else "1"
)

mmsd_test.to_json(
    "idefics2_9B_test_predictions.json",
    orient="records",
    lines=False
)

Processing row 1/2409


/home/gpuuser3/miniconda3/envs/idefics2-sinngam/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:649: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Input=> Question: Classify the text <i am guessing # netflix no longer lets you grab screens of movies . that & the new rating system is so awesome .> and the image into one of the following categories: <SARCASTIC, NOT SARCASTIC>. Answer:
Response=> SARCASTIC

Processing row 2/2409


/home/gpuuser3/miniconda3/envs/idefics2-sinngam/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:649: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Input=> Question: Classify the text <it 's the insensitive strikeouts at suntrust park .> and the image into one of the following categories: <SARCASTIC, NOT SARCASTIC>. Answer:
Response=> NOT SARCASTIC

Processing row 3/2409
Input=> Question: Classify the text <following the path of the river calder , so ... grim ... up .... north> and the image into one of the following categories: <SARCASTIC, NOT SARCASTIC>. Answer:
Response=> SARCASTIC

Processing row 4/2409
Input=> Question: Classify the text <# westernsahara # authority has no lessons 2get from countries of # humanrights high standards> and the image into one of the following categories: <SARCASTIC, NOT SARCASTIC>. Answer:
Response=> SARCASTIC

Processing row 5/2409
Input=> Question: Classify the text <hey <user> great sale !> and the image into one of the following categories: <SARCASTIC, NOT SARCASTIC>. Answer:
Response=> NOT SARCASTIC

Processing row 6/2409
Input=> Question: Classify the text <dad 's handy work . can 't tell a

In [25]:
mmsd_test.head()

,image_id,text,label,image_path,question,idefics2_response,idefics2_label
0,862902619928506372,i am guessing # netflix no longer lets you gra...,1,/home/gpuuser3/sinngam_albert/datasets/mmsd2/d...,Question: Classify the text <i am guessing # n...,SARCASTIC,1
1,892551658487631873,it 's the insensitive strikeouts at suntrust p...,1,/home/gpuuser3/sinngam_albert/datasets/mmsd2/d...,Question: Classify the text <it 's the insensi...,NOT SARCASTIC,0
2,853143461360480256,"following the path of the river calder , so .....",1,/home/gpuuser3/sinngam_albert/datasets/mmsd2/d...,Question: Classify the text <following the pat...,SARCASTIC,1
3,918423568823840768,# westernsahara # authority has no lessons 2ge...,1,/home/gpuuser3/sinngam_albert/datasets/mmsd2/d...,Question: Classify the text <# westernsahara #...,SARCASTIC,1
4,731617467718610944,hey <user> great sale !,1,/home/gpuuser3/sinngam_albert/datasets/mmsd2/d...,Question: Classify the text <hey <user> great ...,NOT SARCASTIC,0
